In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
df= pd.read_csv("C:/Users/farra/Downloads/ufc-fighters-statistics.csv")
df.head()   

,name,nickname,wins,losses,draws,height_cm,weight_in_kg,reach_in_cm,stance,date_of_birth,significant_strikes_landed_per_minute,significant_striking_accuracy,significant_strikes_absorbed_per_minute,significant_strike_defence,average_takedowns_landed_per_15_minutes,takedown_accuracy,takedown_defense,average_submissions_attempted_per_15_minutes
0,Robert Drysdale,NaN,7,0,0,190.50,92.99,NaN,Orthodox,1981-10-05,0.00,0.0,0.00,0.0,7.32,100.0,0.0,21.9
1,Daniel McWilliams,The Animal,15,37,0,185.42,83.91,NaN,NaN,NaN,3.36,77.0,0.00,0.0,0.00,0.0,100.0,21.6
2,Dan Molina,NaN,13,9,0,177.80,97.98,NaN,NaN,NaN,0.00,0.0,5.58,60.0,0.00,0.0,0.0,20.9
3,Paul Ruiz,NaN,7,4,0,167.64,61.23,NaN,NaN,NaN,1.40,33.0,1.40,75.0,0.00,0.0,100.0,20.9
4,Collin Huckbody,All In,8,2,0,190.50,83.91,193.04,Orthodox,1994-09-29,2.05,60.0,2.73,42.0,10.23,100.0,0.0,20.4


In [3]:
df = df.drop(columns=['nickname','date_of_birth'])   

In [4]:
from itertools import combinations
from tqdm import tqdm

df_subset = df.head(5000)



fighters = df.to_dict('records')

pairs = list(combinations(df_subset.index, 2))

data_pairs = []

for pair in tqdm(pairs, desc="Generando pares de luchadores"):
    # Asegúrate de obtener las filas completas de cada luchador
    fighter1 = df.loc[pair[0]]  
    fighter2 = df.loc[pair[1]]  
    
    

    fight_data = {
            'fighter1_name': fighter1["name"],
            'fighter1_wins': fighter1['wins'],
            'fighter1_losses': fighter1['losses'],
            'fighter1_draws': fighter1['draws'],
            'fighter1_height_cm': fighter1['height_cm'],
            'fighter1_weight_in_kg': fighter1['weight_in_kg'],
            'fighter1_reach_in_cm': fighter1['reach_in_cm'],
            'fighter1_stance': fighter1['stance'],
            'fighter2_name': fighter2["name"],
            'fighter2_wins': fighter2['wins'],
            'fighter2_losses': fighter2['losses'],
            'fighter2_draws': fighter2['draws'],
            'fighter2_height_cm': fighter2['height_cm'],
            'fighter2_weight_in_kg': fighter2['weight_in_kg'],
            'fighter2_reach_in_cm': fighter2['reach_in_cm'],
            'fighter2_stance': fighter2['stance'],
            
        }
    data_pairs.append(fight_data)
    #print(df.isnull().sum())

fights_df = pd.DataFrame(data_pairs)
print(fights_df.head())  


Generando pares de luchadores: 100%|██████████| 8448105/8448105 [17:17<00:00, 8141.87it/s]


     fighter1_name  fighter1_wins  fighter1_losses  fighter1_draws  \
0  Robert Drysdale              7                0               0   
1  Robert Drysdale              7                0               0   
2  Robert Drysdale              7                0               0   
3  Robert Drysdale              7                0               0   
4  Robert Drysdale              7                0               0   

   fighter1_height_cm  fighter1_weight_in_kg  fighter1_reach_in_cm  \
0               190.5                  92.99                   NaN   
1               190.5                  92.99                   NaN   
2               190.5                  92.99                   NaN   
3               190.5                  92.99                   NaN   
4               190.5                  92.99                   NaN   

  fighter1_stance      fighter2_name  fighter2_wins  fighter2_losses  \
0        Orthodox  Daniel McWilliams             15               37   
1        Ortho

In [5]:
fights_df = pd.get_dummies(fights_df, columns=['fighter1_stance', 'fighter2_stance'], drop_first=True)

In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


df2 = (fights_df)  


df2['fighter1_reach_in_cm'].fillna(df2['fighter1_reach_in_cm'].mean(), inplace=True)
df2['fighter2_reach_in_cm'].fillna(df2['fighter2_reach_in_cm'].mean(), inplace=True)


#df2['fighter1_stance'].fillna(df2['fighter1_stance'].mode()[0], inplace=True)
#df2['fighter2_stance'].fillna(df2['fighter2_stance'].mode()[0], inplace=True)


#df2 = pd.get_dummies(df2, columns=['fighter1_stance', 'fighter2_stance'], drop_first=True)

# Crear la variable objetivo (1 si fighter1 gana, 0 si fighter2 gana)

df2['result'] = (df2['fighter1_wins'] > df2['fighter2_wins']).astype(int)


X = df2.drop(columns=['fighter1_name', 'fighter2_name', 'result'])  # Quitar nombres y resultado
y = df2['result']  # La columna que indica quién ganó

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [7]:
from sklearn.ensemble import HistGradientBoostingClassifier

clf = HistGradientBoostingClassifier()
clf.fit(X_train, y_train)
y_train_pred = clf.predict(X_train)
print('accuracy_score: ' , accuracy_score(y_train, y_train_pred))
y_test_pred= clf.predict(X_test)
print('accuracy_score: ' , accuracy_score(y_test, y_test_pred))


accuracy_score:  0.99999797080427
accuracy_score:  0.9999964489084734


In [8]:
import pickle 
with open('clf','wb') as file:
    pickle.dump(clf,file)
    